This script aims to train our autoencoder on the clean training data that have been previously generated.

Deepfake is implemented as an autoencoder. 

The face of the actor 1 is cropped and aligned to his face. Then the autoencoder learn how to encode and decode (reconstruct) the face. The goal here is to minimize the reconstruction error.


<img src="image/encoder1.JPG">

The face of the actor 2 is cropped and aligned to his face. Then the autoencoder learn how to encode and decode (reconstruct) the face. The goal here is to minimize the reconstruction error. The interesting part is that the encoder is the same for actor 1 and 2.

<img src="image/encoder2.JPG">

The goal is to find a mathematical function that for a face of the actor 1, outputs a face that looks like the actor 2.

Note: these images don t belong to me and comes from the animation in the youtube video about deepfake made by Siraj Raval

In contrast to the base version of Deepfakes, we provide a very conviniant way to speed up the training process via transfer learning. The idea is to load the weight of a pretrained network (for the same network configuration but for another face pair such as Donald ump and Nicolas Cage) These weights are used as a starting point to learn new weight. The netork will therefore converge faster thant if it would be trained from scratch. 

More advanced technique exists to apply transfer learning to autoencoder such [here](https://www.ijcai.org/Proceedings/15/Papers/578.pdf)

We load the various dependencies

In [0]:
!mkdir saved_model

#Download auxilliary components first


!gdown https://drive.google.com/uc?id=1O0jrWmtAoSN-W8AwmO0GrqhcYPzkZB2Y
  
!unzip deepfake_aux.zip

#Download extracted datasets from pt.1
#Tom

!gdown https://drive.google.com/uc?id=1U7WkAUzGFuSIil2NvaZd3dpCzn34BwlQ
#Nic

!gdown https://drive.google.com/uc?id=1txJSjz3GYjDm9OgMI-YTW8yWvrzVJED3
#ChrisOLD
!gdown https://drive.google.com/uc?id=1Dpm6QssNM7K3vl_-9n9YrIBqLWFi1OFx
  
!unzip extracted_tom.zip
!unzip extracted_nic.zip
!unzip extracted_chris.zip
  

Downloading...
From: https://drive.google.com/uc?id=1O0jrWmtAoSN-W8AwmO0GrqhcYPzkZB2Y
To: /content/deepfake_aux.zip
100% 906k/906k [00:00<00:00, 60.4MB/s]
Archive:  deepfake_aux.zip
   creating: filter/
  inflating: filter/000001.jpg       
  inflating: filter/chrisfilter.jpg  
  inflating: filter/nicfilter.jpg    
  inflating: filter/tomfilter.jpg    
  inflating: filter/tomfilter2.jpg   
   creating: lib_1/
  inflating: lib_1/aligner.py        
  inflating: lib_1/Convert_Adjust.py  
  inflating: lib_1/Convert_Masked.py  
  inflating: lib_1/Extract_Align.py  
  inflating: lib_1/Extract_Crop.py   
  inflating: lib_1/FaceFilter.py     
  inflating: lib_1/faces_detect.py   
  inflating: lib_1/ModelAE.py        
  inflating: lib_1/Model_LowMem.py   
  inflating: lib_1/Model_Original.py  
  inflating: lib_1/PixelShuffler.py  
  inflating: lib_1/PluginLoader.py   
  inflating: lib_1/training_data.py  
  inflating: lib_1/umeyama.py        
  inflating: lib_1/utils.py          
Downloading...

In [0]:
#Install dependencies if missing

!pip install face_recognition

!pip install scandir
!pip install h5py
!pip install opencv-python
!pip install scikit-image
!pip install dlib
!pip install tqdm

#Past 2.2.0, Keras's normalize_data_format method was moved to each specific layer lib.
#For convenience, we'll just use 2.2.0
!pip install --force-reinstall keras==2.2.0



     |████████████████████████████████| 100.2MB 259kB/s 
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models
  Stored in directory: /root/.cache/pip/wheels/91/95/75/19c98a91239878abbc7c59970abd3b4e0438a7dd5b61778335
Successfully built scandir
     |████████████████████████████████| 307kB 4.9MB/s 
     |████████████████████████████████| 17.3MB 39.6MB/s 
     |████████████████████████████████| 2.8MB 31.9MB/s 
     |████████████████████████████████| 51kB 19.9MB/s 
     |████████████████████████████████| 25.2MB 1.3MB/s 
     |████████████████████████████████| 276kB 42.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built pyyaml
ERROR: tensorflow 1.14.0 has requirement keras-applications>=1.0.6, but you'll have keras-applications 1.0.2 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement keras-preprocessin

In [0]:
#Download pretrained weight to speed convergence. Weights2 from FakeAPP

#!gdown https://drive.google.com/uc?id=1ir6KEzNp63ZzWwIDjKcO7hgpuKOE7sil 

!gdown https://drive.google.com/uc?id=1biKvZSxV-Dx9mUX-UMaFcZ6J8OMQ9MHQ
!unzip weights2.zip

Downloading...
From: https://drive.google.com/uc?id=1biKvZSxV-Dx9mUX-UMaFcZ6J8OMQ9MHQ
To: /content/weights2.zip
304MB [00:02, 107MB/s] 
Archive:  weights2.zip
   creating: weight/
  inflating: weight/decoder_A.h5     
  inflating: weight/decoder_B.h5     
  inflating: weight/encoder.h5       


In [0]:
import cv2
import numpy
import time

from pathlib import Path
from scandir import scandir

from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Flatten, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam

from lib_1.PixelShuffler import PixelShuffler
import time
import numpy
from lib_1.training_data import minibatchAB, stack_images




Using TensorFlow backend.


We define the caracteristics of our network:
   * sav_Model: the location of the model (it needs to be the absolute path)
   * pretrained_weight: transfer learning can be used to speed up the learning process
   * sav_Model: the location of the model (it needs to be the absolute path)
   * image_actor_A_directory: the location of the training data for the actor A
   * image_actor_B_directory: the location of the training data for the actor B
   * batch_size: the batch size (1<< batch_size <<your training dataset size)
       * a small batch_size enables poor hardware to run the training but the gradiant might be a bit noisy. I advice you to pick a batch size of 32 or 64
   * save_interval: it define at which interval of time Keras should save your model
   * IMAGE_SHAPE: the shape of the input image
       * it must be consistant with the one you use during the prediction
   * ENCODER_DIM: the dimension of the encoding in the autoencoder
       * it must be consistant with the one you use during the prediction
   
I invite you to try different parameter until you get a satifying result.

In [0]:
#Define all directories

sav_Model="../content/saved_model/"
pretrained_weight="../content/weight"
image_actor_A_directory="../content/chris" #ORIGINAL
image_actor_B_directory="../content/nic" #TARGET TO REPLACE WITH
batch_size=1
save_interval=100
ENCODER_DIM = 1024


#DON'T MODIFY
image_extensions = [".jpg", ".jpeg", ".png"]
encoderH5 = '/encoder.h5'
decoder_AH5 = '/decoder_A.h5'
decoder_BH5 = '/decoder_B.h5'
IMAGE_SHAPE = (64, 64, 3)


We define the model and its associated trainer.

In [0]:
class dfModel():
    def __init__(self):

        self.model_dir = sav_Model
        self.pretrained_weight=pretrained_weight
        self.encoder = self.Encoder()
        self.decoder_A = self.Decoder()
        self.decoder_B = self.Decoder()

        self.initModel()

    
    def initModel(self):
        optimizer = Adam(lr=5e-5, beta_1=0.5, beta_2=0.999) #orig adam 5e-5
        x = Input(shape=IMAGE_SHAPE)

        self.autoencoder_A = KerasModel(x, self.decoder_A(self.encoder(x)))
        self.autoencoder_B = KerasModel(x, self.decoder_B(self.encoder(x)))
        print(self.encoder.summary())
        print(self.decoder_A.summary())


        self.autoencoder_A.compile(optimizer=optimizer, loss='mean_absolute_error')
        self.autoencoder_B.compile(optimizer=optimizer, loss='mean_absolute_error')

    def converter(self, swap):
        autoencoder = self.autoencoder_B if not swap else self.autoencoder_A 
        return lambda img: autoencoder.predict(img)

    def conv(self, filters):
        def block(x):
            x = Conv2D(filters, kernel_size=5, strides=2, padding='same')(x)
            x = LeakyReLU(0.1)(x)
            return x
        return block

    def upscale(self, filters):
        def block(x):
            x = Conv2D(filters * 4, kernel_size=3, padding='same')(x)
            x = LeakyReLU(0.1)(x)
            x = PixelShuffler()(x)
            #Pixelshufflers job here is analoguous to upsampling2d
            return x
        return block
#Note how no maxpooling after every layer here, we are generating a WARPED image, not a lower dimensional representation first.
    def Encoder(self):
        input_ = Input(shape=IMAGE_SHAPE)
        x = input_
        x = self.conv(128)(x)
        x = self.conv(256)(x)
        x = self.conv(512)(x)
        x = self.conv(1024)(x)
        x = Dense(ENCODER_DIM)(Flatten()(x))
        x = Dense(4 * 4 * 1024)(x)
        #Passed flattened X input into 2 dense layers, 1024 and 1024*4*4
        x = Reshape((4, 4, 1024))(x)
        #Reshapes X into 4,4,1024
        x = self.upscale(512)(x)
        return KerasModel(input_, x)

    def Decoder(self):
        input_ = Input(shape=(8, 8, 512))
        x = input_
        x = self.upscale(256)(x) #Actually 1024 given filters*4
        x = self.upscale(128)(x) #Actually 512
        x = self.upscale(64)(x) #Actually 256
        x = Conv2D(3, kernel_size=5, padding='same', activation='sigmoid')(x)
        return KerasModel(input_, x)
        
    def load(self, swapped):
        (face_A,face_B) = (decoder_AH5, decoder_BH5) if not swapped else (decoder_BH5, decoder_AH5)

        try:
            self.encoder.load_weights(self.pretrained_weight + encoderH5)
            self.decoder_A.load_weights(self.pretrained_weight + face_A)
            self.decoder_B.load_weights(self.pretrained_weight + face_B)
            print('loaded model weights')
            return True
        except Exception as e:
            print('Failed loading existing training data.')
            print(e)
            return False

    def save_weights(self):
        self.encoder.save_weights(self.model_dir + encoderH5)
        self.decoder_A.save_weights(self.model_dir + decoder_AH5)
        self.decoder_B.save_weights(self.model_dir + decoder_BH5)
        print('saved model weights')

class Trainer():
    def __init__(self, model, fn_A, fn_B, batch_size=64):
        self.batch_size = batch_size
        self.model = model
        self.images_A = minibatchAB(fn_A, self.batch_size)
        self.images_B = minibatchAB(fn_B, self.batch_size)

    def train_one_step(self, iter):
        epoch, warped_A, target_A = next(self.images_A)
        epoch, warped_B, target_B = next(self.images_B)

        loss_A = self.model.autoencoder_A.train_on_batch(warped_A, target_A)
        loss_B = self.model.autoencoder_B.train_on_batch(warped_B, target_B)
        print("[{0}] [#{1:05d}] loss_A: {2:.5f}, loss_B: {3:.5f}".format(time.strftime("%H:%M:%S"), iter, loss_A, loss_B),
            end='\r')


In [0]:
def get_image_paths(directory):
    return [x.path for x in scandir(directory) if
     any(map(lambda ext: x.name.lower().endswith(ext), image_extensions))]

We launch the training. We will try to minimize th loss of the auto encoder A and B.

In [0]:

model = dfModel()
model.load(swapped=False)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 32, 32, 128)       9728      
_________________________________________________________________
leaky_re_lu_45 (LeakyReLU)   (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_46 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 8, 8, 512)         3277312   
_________________________________________________________________
leaky_re_lu_47 (LeakyReLU)   (None, 8, 8, 512)         0         
__________

True

In [0]:
print('Loading data, this may take a while...')
# this is so that you can enter case insensitive values for trainer
#1407 STOPPED AT 63 EPOCHS Disabled load to do it ourselves

model = dfModel()
model.load(swapped=False)



images_A = get_image_paths(image_actor_A_directory)
images_B = get_image_paths(image_actor_B_directory)
trainer = Trainer(model,images_A,images_B,batch_size=batch_size)

for epoch in range(0, 1000000):

    save_iteration = epoch % save_interval == 0

    trainer.train_one_step(epoch)

    if save_iteration:
        model.save_weights()
        
    